<a href="https://colab.research.google.com/github/XinpeiMa/My_NoteBook/blob/master/APM_DataProject_XinpeiMa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (0) Data pre-processing
- adjust datatype
- sort date
- count unique tickers

In [0]:
import pandas as pd
import numpy as np
import random
from functools import reduce
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.optimize import minimize
random.seed(30)
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")
plt.figure(figsize=(15,9))
%pylab inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Parameters
path = "drive/My Drive/My Classes/APM/DataProject/prices.csv"
annual_r = 0.015
r = (annual_r + 1) ** (1. / 365.) - 1. # daily risk-less rate of return

In [0]:
def change_str_to_float(x):
    if x == ".":
        return 0.
    return float(x)

In [0]:
df = pd.read_csv(path, header=0)
df = df.loc[df.PRC != 'PRC', :]
df['TICKER'] = df['TICKER'].astype(str)
df['date'] = df['date'].astype(str)
df['PRC'] = df['PRC'].apply(lambda x: change_str_to_float(x))
df['VOL'] = df['VOL'].apply(lambda x: change_str_to_float(x))
tickers = df.TICKER.unique().tolist()
df.set_index('date', inplace=True)
df.date = df.index
df['Date'] = pd.to_datetime(df.date)
df.sort_values(by='Date')
print(df.TICKER.unique().tolist())

## (1) compute $v_t$ signal

#### Rolling window out-of-sample forecasting

- In order to compute $\beta$, I implemented a rolling window out-of-sample approach that using a fixed number of the most recent data（i.e., 66 days) at each point of time. I examined the variance and bias value of applying different window size. Finally, I found that window size 66 yielded the best result (with minimal total error). 

- The beta that we get from regressing of asset's returns on the returns of the market represents the sensitivity of an asset's return stream to market-wide shocks, therefore $v_t$ represents a signal of stock idiosyncratic risk signal.


In [0]:
def calc_residual_reversion_signal(tk, df, window_size, r):
    SPY = df.loc[df['TICKER'] == 'SPY', ['PRC', 'VOL']]
    SPY.columns = ["SPY_PRC", "SPY_VOL"]
    Stock = df.loc[df['TICKER'] == tk, ['PRC', 'VOL']]
    merged = pd.merge(left=SPY, right=Stock, right_index=True, left_index=True, how='outer')
    merged['stock_ret'] = np.log(merged.PRC) - np.log(merged.PRC.shift(1))
    merged['market_ret'] = np.log(merged.SPY_PRC) - np.log(merged.SPY_PRC.shift(1))
    cov = merged.stock_ret.rolling(window_size).cov(merged.market_ret.rolling(window_size))
    var = merged.market_ret.rolling(window_size).var()
    merged['beta'] = cov / var
    merged['temp'] = merged['stock_ret'] - merged['market_ret'] * merged['beta']
    merged['vt'] = merged['temp'].shift(1)
    merged.drop(columns=['temp'], inplace=True)
    return merged

#### Find Optimal Window Size

In [0]:
windows = [22, 66, 132, 154, 176, 225, 500]
bias_list = []
variance_list = []
for i in range(len(windows)):
    print("Windows {} days".format(windows[i]))
    bias_res = 0.0
    var_res = 0.0
    count = 0
    for ticker in tickers:
        if ticker != 'SPY':
            residual_reversion_df = calc_residual_reversion_signal(ticker, df, windows[i], r)
            residual_reversion_df = residual_reversion_df.loc[residual_reversion_df['vt'].notnull(), :]

            pred = residual_reversion_df['market_ret'] * residual_reversion_df['beta']
            real = residual_reversion_df['stock_ret']
            variance = np.mean(np.square(pred - pred.mean()))
            # sum(real - E[predictced])^2
            bias = np.mean(np.square(real - pred.mean()))
            # sum(predicted - E[predictced])^2
            if np.isfinite(variance) and np.isfinite(bias):
                bias_res += bias
                var_res += variance
                count += 1

    variance_list.append(var_res/count)
    bias_list.append(bias_res/count)
    

In [0]:
print(bias_list, variance_list)
plot_df = pd.DataFrame({"bias": bias_list, "variance": variance_list}, index=[str(x) + "d" for x in windows])
plot_df['total_error'] = plot_df.bias + plot_df.variance
print(plot_df)
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(plot_df.bias, 'o-', linewidth=2, markersize=6, label="bias", alpha=0.5)
ax.plot(plot_df.variance, 'o-', linewidth=2, markersize=6, label='variance', alpha=0.5)
ax.plot(plot_df.total_error, 'o-', linewidth=2, markersize=6, label='total error')
ax.plot(1, plot_df.total_error.min(), marker='o', markersize=6, color="red", label='minimal')
ax.set_xticklabels(plot_df.index.tolist())
ax.legend()
plt.show()

From the above experiment, we select rolling window size 132 days, because it seems like the best variance-bias trade-off.

In [0]:
rolling_window = 66

#### An example of $v_t$ signal computed using 66 days rolling window

In [0]:
stock_signal_df = calc_residual_reversion_signal("IBM", df, rolling_window, r)
stock_signal_df.to_csv("K:\My Drive\PythonProjects\APMDataProject\ibm.csv")
display(stock_signal_df)

#### Does $v_t^1$ predict $r_{t, t+1}^i$ and $r_{t+1, t+2}^i$ etc.?
I checked the $R^2$ and $p$ value between $v_t^1$ and $r_{t, t+1}^i$ and the  $R^2$ and $p$ between $v_t^1$ and $r_{t+1, t+2}^i$

In [0]:
def measure_predictive_capacity(df, x, y, time_lag):
	# if paired x, y if any value in x or y is none then exclude it
	def remove_nan(x, y):
		pairs = []
		for i in range(len(x)):
			if np.isnan(x[i]) or np.isnan(y[i]):
				continue
			pairs.append((x[i], y[i]))
		return pairs
		
    # test on out-of-sample part
	out_sample = df.loc[df[y].notnull(), :]
	out_sample['shift'] = out_sample.loc[:, x].shift(periods=time_lag)
	xy = remove_nan(np.array(out_sample['shift'].values, dtype=np.float64), 
                    np.array(out_sample[y].values, dtype=np.float64))
	if len(xy) > 10:
		model = sm.OLS([i[0] for i in xy], [i[1] for i in xy])
		results = model.fit()
		return results.rsquared, results.pvalues[0]
	return None, None

In [0]:
df_map = {}
for ticker in tickers:
    stock_signal_df = calc_residual_reversion_signal(ticker, df, rolling_window, r)
    df_map[ticker] = stock_signal_df
print(df_map.keys())

In [0]:
time_lag_1, time_lag_2 = -1, -2
time_lag_dependency_results = {}

for ticker in tickers:
    if ticker != 'SPY':
        res = {}
        stock_signal_df = df_map[ticker]
        r_square_1, p_value_1 = measure_predictive_capacity(stock_signal_df, 'stock_ret', 'vt', time_lag_1)
        r_square_2, p_value_2 = measure_predictive_capacity(stock_signal_df, 'stock_ret', 'vt', time_lag_2)
        
        if len((stock_signal_df['beta'].values.tolist())) > 1:
            res['beta'] = stock_signal_df['beta'].values.tolist()[-1]
            res['lag_1_rsquare'] = r_square_1
            res['lag_1_pvalue'] = p_value_1
            res['lag_2_rsquare'] = r_square_2
            res['lag_2_pvalue'] = p_value_2

        time_lag_dependency_results[ticker] = res
    

data = []
for k, v in time_lag_dependency_results.items():
    data.append([k] + list(v.values()))
res = pd.DataFrame(data,  columns = ['ticker', 'beta', '1_step_rsquare', '1_step_p', '2_step_rsquare', '2_step_p']) 
res = np.round(res, decimals=2)

print(time_lag_dependency_results)

In [0]:
print(res.loc[:, ['1_step_rsquare', '1_step_p', '2_step_rsquare', '2_step_p']].describe())

- From the statistic results, I found that the generated signal $v_t$ doesnt have significantly association between future returns (lag 1 and lag 2) based on the observation that the average $p$ value is very high (around 0.2). 
- The association between 
 $v_t$ and $r_{t+1}$ is usually very close to the association between $v_t$ and $r_{t+2}$. Slideghtly larger $R^2$ of $v_t$ and $r_{t+2}$ indicate the mean-reverting pattern.

## (2) Create the corresponding Markowitz portfolio and track its performance
Basic Assumptions: 
- We assumed that investors are able to trade without delay or cost.
- We assumed that all investors are "mean-variance optimizers". What this essentially means is that they would only 
demand portfolios that have the highest return attainable for a given level of risk. 
- Here, I only consider long only portfolio in order to simplify borrow/lend procedure.

Data Preparation for Portfolio Construction

In [0]:
selected_df_list = []
for ticker in df_map.keys():
    temp_df = df_map[ticker]
    temp_df_vt = temp_df.loc[temp_df['vt'].notnull(), ['stock_ret', 'beta', 'vt', 'PRC', 'VOL']].copy()
    temp_df_vt = temp_df_vt.loc[~temp_df_vt.index.duplicated(keep='first')]
    temp_df_vt.columns = [ticker + '_ret', ticker + '_beta', ticker + '_vt', ticker + "_prc", ticker + "_vol"]
    selected_df_list.append(temp_df_vt)
vt_df = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True, how='outer'), selected_df_list)
print(vt_df.columns)
print("{} of stocks are included".format(len(selected_df_list)))

In [0]:
SPY_var = df.loc[df.TICKER == "SPY", 'PRC'].rolling(rolling_window).var()
SPY_mean = df.loc[df.TICKER == "SPY", 'PRC'].rolling(rolling_window).mean()
SPY_ = pd.concat([SPY_mean, SPY_var], axis=1)
SPY_.columns = ['SPY_mean', 'SPY_var']

In [0]:
portfolio_prep_df = vt_df.merge(SPY_, left_index=True, right_index=True, how='left')
prep_cols = portfolio_prep_df.columns
for col in prep_cols:
    if col[-3:] == '_vt':
        portfolio_prep_df[col + '_var'] = portfolio_prep_df[col].rolling(rolling_window).var()
portfolio_prep_df = portfolio_prep_df.drop_duplicates()

In [0]:
all_dates = portfolio_prep_df.index.unique().tolist()

Based on our slides, we compute the risk tolerance as
$\kappa_M = \frac{\mathbb{E}R_M -  \mu_0}{\mathbb{Var} R_M}$.
Weights $w$ satisfies constrained $\sum_i{w_i} =1$ and $w_i \in [0, 1]$.


In [0]:
def compute_covariance(date, prep_df):
    vt_vars, betas, returns, included_tickers = [], [], [], []

    for tk in df_map.keys():
        try:
            beta, vt_var, ret = prep_df.at[date, tk + '_beta'], \
                                prep_df.at[date, tk + '_vt_var']/rolling_window, \
                                prep_df.at[date, tk + '_ret']
            if np.isfinite(beta) and np.isfinite(vt_var) and np.isfinite(ret):
                betas.append(beta)
                vt_vars.append(vt_var)
                returns.append(ret)
                included_tickers.append(tk)
        except:
            continue

    betas = np.array(betas).reshape(-1, 1)
    SPY_var = prep_df.at[date, 'SPY_var'] / rolling_window
    cov = betas.dot(betas.transpose()) * SPY_var + np.diag(np.array(vt_vars))
    return [cov, np.array(returns), included_tickers]

In [0]:
def get_markovitz_weights(date, prep_df, r):
    cov, rets, tks = compute_covariance(date, prep_df)
    risk_adjusted_returns = np.array(rets) - np.ones(len(rets)) * r
    SPY_var = prep_df.at[date, 'SPY_var'] / rolling_window
    kappa = (prep_df.at[date, 'SPY_mean'] - r)/ SPY_var
    def objective(w):
        return 0.5 * kappa * w.transpose().dot(cov).dot(w) - np.dot(w, risk_adjusted_returns) 

    w0 = [1.0/len(risk_adjusted_returns) for _ in range(len(risk_adjusted_returns))]
    e = np.ones(len(risk_adjusted_returns))
    bnds = [(0., 1.) for _ in range(len(risk_adjusted_returns))]
    const = ({'type': 'eq', 'fun': lambda w: np.dot(w, e) - 1.})
    try:
        solution = minimize(fun=objective, x0=w0, method='SLSQP', bounds=bnds, constraints=const)
        w = solution.x.round(6)
    except:
        w = None
    return tks, w

In [0]:
def get_next_day(date):
    curr_index = all_dates.index(date)
    return all_dates[curr_index + 1] if curr_index + 1 < len(all_dates) else None

In [0]:
def calc_portfolio_ret(date, weights, tks, df):
    rets = []
    for tk in tks:
        rets.append(df.at[date, tk + "_ret"])
    portfolio_return = np.array(rets).dot(weights)
    return portfolio_return

In [0]:
def MDD(xs):
''' maximum drawdown '''   
    i = np.argmax(np.maximum.accumulate(xs) - xs) # end of the period
    j = np.argmax(xs[:i]) # start of period
    return abs(xs[i]-xs[j])

In [0]:
def track_portfolio_performance(start_date_index, end_date_index):
    pnl = 100.
    ret = None
    portfolio_history = {}
    pnl_list = [100.]
    for i in range(start_date_index, end_date_index):
        curr_day = all_dates[i]
        next_day = get_next_day(curr_day)
        markovitz_rs = get_markovitz_weights(curr_day, portfolio_prep_df, r)

        if markovitz_rs[1] is not None:
            curr_tickers, curr_weights = markovitz_rs[0], markovitz_rs[1]
            ret = calc_portfolio_ret(next_day, curr_weights, curr_tickers, portfolio_prep_df)
            if i == start_date_index:
                portfolio_history[curr_day] = [ret, pnl, 0.]
        if not np.isfinite(ret):
            ret = 0.
        pnl *= np.exp(ret)
        pnl_list.append(pnl)
        try:
            portfolio_history[next_day] = [ret, pnl, MDD(pnl_list)]
        except:
            portfolio_history[next_day] = [ret, pnl, 0]

    return portfolio_history

Since we have full out-of-sample data for 2009, 2010 and 2011 due to the rolling window approach. Data for 2008 is not quite complete due to the rolling window procedure.
We keep track of the monthly portfolio for the three years.

In [0]:
def find_montly_index(year):
    monthly_index = {}
    for i in range(len(all_dates)):
        if all_dates[i][:4] == year:
            if all_dates[i][4:6] not in monthly_index:
                monthly_index[all_dates[i][4:6]] = []
            monthly_index[all_dates[i][4:6]].append(i)
    return monthly_index

In [0]:
index_2009 = find_montly_index('2009')
print(index_2009)
res_2009 = dict()
for mo in index_2009.keys():
    start, end = min(index_2009[mo]), max(index_2009[mo])
    portfolio = track_portfolio_performance(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", 'mmd']
    res_2009[mo] = []
    res_2009[mo].append(portfolio_df['ret'].mean() - r)
    res_2009[mo].append(portfolio_df['ret'].std()/np.sqrt(len(index_2009[mo])))
    res_2009[mo].append(portfolio_df['mmd'].max())

In [97]:
df_2009_1 = pd.DataFrame(res_2009, index=['average excess returns', 'vol', 'mmd']).T
df_2009_1['sharpe ratio'] = df_2009_1['average excess returns'] / df_2009_1['vol']
display(df_2009_1)
display(df_2009_1.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.579969,0.151852,2.300242,3.819312
02,0.599056,0.100263,0.000000,5.974827
03,0.814004,0.129195,0.000000,6.300593
04,0.185003,0.127062,28.830281,1.456007
05,0.001815,0.003053,4.610837,0.594688
06,0.266027,0.118339,4.073553,2.247997
07,0.328838,0.112796,2588.760492,2.915347
08,0.001944,0.003631,4.789287,0.535493
09,0.001999,0.002052,3.545142,0.974331
10,0.078771,0.076717,8.289380,1.026775


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,12.000000,12.000000
mean,0.244311,0.072148,221.506930,2.357648
std,0.280621,0.058446,745.530035,2.034912
min,0.001492,0.002052,0.000000,0.530647
25%,0.001986,0.003486,3.233917,0.879420
50%,0.131887,0.088490,4.606542,1.685883
75%,0.391621,0.120520,8.283617,3.141339
max,0.814004,0.151852,2588.760492,6.300593


In [0]:
index_2010 =  find_montly_index('2010')
res_2010 = dict()
for mo in index_2010.keys():
    start, end = min(index_2010[mo]), max(index_2010[mo])
    portfolio = track_portfolio_performance(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", 'mmd']
    res_2010[mo] = []
    res_2010[mo].append(portfolio_df['ret'].mean() - r)
    res_2010[mo].append(portfolio_df['ret'].std())
    res_2010[mo].append(portfolio_df['mmd'].max())

In [100]:
df_2010_1 = pd.DataFrame(res_2010, index=['average excess returns', 'vol', 'mmd']).T
df_2010_1['sharpe ratio'] = df_2010_1['average excess returns'] / df_2010_1['vol']
display(df_2010_1)
display(df_2010_1.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.100665,0.188038,4.917541,0.535342
02,0.364856,0.177507,0.000000,2.055448
03,0.164005,0.237528,5.448070,0.690466
04,0.057889,0.036113,1.688045,1.602993
05,0.544171,0.787888,65.577764,0.690671
06,0.297073,0.149229,0.000000,1.990718
07,0.640515,1.231943,825335.503709,0.519923
08,0.098921,0.200674,3.148638,0.492943
09,0.415016,1.448861,16356.995460,0.286443
10,0.052936,0.038108,2.027311,1.389102


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,12.000000,12.000000
mean,0.233201,0.381124,70148.429622,0.988438
std,0.213737,0.494164,237868.684789,0.619752
min,0.021103,0.036113,0.000000,0.286443
25%,0.056651,0.038802,1.655338,0.531487
50%,0.132335,0.182772,3.720176,0.690569
75%,0.377396,0.375118,20.480494,1.442575
max,0.640515,1.448861,825335.503709,2.055448


In [0]:
index_2011 =  find_montly_index('2011')
print(index_2011)
res_2011 = dict()
for mo in index_2011.keys():
    start, end = min(index_2011[mo]), max(index_2011[mo])
    portfolio = track_portfolio_performance(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", 'mmd']
    res_2011[mo] = []
    res_2011[mo].append(portfolio_df['ret'].mean() - r)
    res_2011[mo].append(portfolio_df['ret'].std()/np.sqrt(len(index_2011[mo])))
    res_2011[mo].append(portfolio_df['mmd'].max())

In [103]:
df_2011_1 = pd.DataFrame(res_2011, index=['average excess returns', 'vol', 'mmd']).T
df_2011_1['sharpe ratio'] = df_2011_1['average excess returns'] / df_2011_1['vol']
display(df_2011_1)
display(df_2011_1.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.068662,0.005571,0.000000e+00,12.324469
02,0.062044,0.007749,1.201352e+00,8.007226
03,0.624013,0.139111,0.000000e+00,4.485710
04,0.638935,0.161018,0.000000e+00,3.968086
05,0.222071,0.081479,6.026495e-01,2.725489
06,0.105015,0.020075,2.388797e+00,5.231095
07,0.679151,0.333670,3.740290e+06,2.035397
08,0.474814,0.102887,0.000000e+00,4.614923
09,0.656541,0.121138,4.159798e+04,5.419764
10,0.270430,0.154576,5.210281e+02,1.749488


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,1.200000e+01,12.000000
mean,0.413475,0.139696,2.267245e+07,4.523077
std,0.296569,0.118685,7.735597e+07,3.114710
min,0.062044,0.005571,0.000000e+00,1.237305
25%,0.176272,0.066128,0.000000e+00,2.367333
50%,0.372622,0.130125,1.795075e+00,4.226898
75%,0.643336,0.161179,1.079027e+04,5.278262
max,0.960004,0.387414,2.682870e+08,12.324469


## (3) Consider Market Impact

- Define market impact model
$\text{tcost} = \alpha \left| \Delta w_t \right|^2 +  \beta \sigma \sqrt{\frac{Q}{T} }$
where  $Q$ is the number of shares to be traded, $V$ is daily volume. Both $\alpha$ and $\beta$ are two constant parameters, $\alpha$ represents the fixed cost per share, and $\beta$ is the coefficient expressing the liquidity of the stock.

 
- The objective function can be formulated as
$\max_w \mathbb{E}(w R) - \frac{\kappa}{2} \mathbb{Var} (w R) - \alpha \left| \Delta w_t \right|^2 +  \beta \sigma \sqrt{\frac{Q}{T} }$. The first tem is expected returns, the second term is risk-aversion, the third term is a fixed cost for Bid/Ask spread and the last term is a dynamic term of market impact. In general the last two terms serve as important component of trading cost.

- Based on our knowledge about around 10 bps is the average impact cost for US large-cap stocks. We can guess the value of $\alpha$ and $\beta$.


In [0]:
def get_total_market_volume(date, tks, prep_df):
    overall_volume = 0.0
    for tk in tks:
        overall_volume += max(abs(prep_df.at[date, tk + '_prc'] * prep_df.at[date, tk + '_vol']), 0.0)
    return overall_volume

In [0]:
def get_weight_change(w, prev_w):
    weight_delta = 0.0
    n = len(w)
    if prev_w is None:
        for i in range(n):
            weight_delta += abs(w[i])
    else:
        for i in range(n):
            try:
                if np.sign(w[i]) == np.sign(prev_w[i]):
                    weight_delta += abs(w[i] - prev_w[i])
                else:
                    weight_delta += abs(w[i]) + abs(prev_w[i])
            except:
                pass
    return weight_delta

In [0]:
def markovitz_plus_market_impact_weights(date, prep_df, r, pnl, prev_w):
    # all avaiable trading for curr date
    cov, rets, tks = compute_covariance(date, prep_df) 
    adj_returns = np.array(rets) - np.ones(len(rets)) * r
    SPY_var = prep_df.at[date, 'SPY_var'] / rolling_window
    kappa = (prep_df.at[date, 'SPY_mean'] - r) / SPY_var


    def objective(w):
        weight_change = get_weight_change(w, prev_w)
        market_impact = weight_change ** 2 + 100 * np.sqrt((pnl * weight_change) / get_total_market_volume(date, tks, prep_df))
        return 0.5 * kappa * w.transpose().dot(cov).dot(w) + market_impact - np.dot(w, adj_returns)

    w0 = [1.0 / len(rets) for _ in range(len(rets))]
    e = np.ones(len(rets))
    bnds = [(0, 1) for _ in range(len(rets))]
    const = ({'type': 'eq', 'fun': lambda w: np.dot(w, e) - 1.})
    solution = minimize(fun=objective, x0=w0, method='SLSQP', bounds=bnds, constraints=const)
    w = solution.x.round(6)

    return w, tks



def calc_portfolio_returns(date, w, tks, df):
    rets = []
    for tk in tks:
        try:
            rets.append(df.at[date, tk + "_ret"])
        except:
            rets.append(0.0)
    portfolio_return = np.array(rets).dot(w)
    return portfolio_return



def track_portfolio_performance_1(start, end, prep_df=portfolio_prep_df, daily_r=r):
    pnl = 100.
    portfolio_history = {}
    prev_w = None
    pnl_list =[100.]

    for i in range(start, end):
        curr_day = all_dates[i]
        next_day = get_next_day(curr_day)

        if i == start:
            portfolio_history[curr_day] = [0.0, pnl]

        w, tks = markovitz_plus_market_impact_weights(next_day, prep_df, daily_r, pnl, prev_w)
        portfolio_ret = calc_portfolio_returns(next_day, w, tks, prep_df)
        pnl += pnl * portfolio_ret
        pnl_list.append(pnl)
        try:
            portfolio_history[next_day] = [portfolio_ret, pnl, MDD(pnl_list)]
        except:
            portfolio_history[next_day] = [portfolio_ret, pnl, 0]
        prev_w = w

    return portfolio_history

In [0]:
index_2009 =  find_montly_index('2009')
res_2009_mi = dict()
for mo in index_2010.keys():
    start, end = min(index_2009[mo]), max(index_2009[mo])
    portfolio = track_portfolio_performance_1(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", "mmd"]
    res_2009_mi[mo] = []
    res_2009_mi[mo].append(portfolio_df['ret'].mean() - r)
    res_2009_mi[mo].append(portfolio_df['ret'].std())
    res_2009_mi[mo].append(portfolio_df['mmd'].max())


In [110]:
df_2009_2 = pd.DataFrame(res_2009_mi, index=['average excess returns', 'vol', 'mmd']).T
df_2009_2['sharpe ratio'] = df_2009_2['average excess returns'] / df_2009_2['vol']
display(df_2009_2)
display(df_2009_2.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.398021,0.430404,0.887094,0.924763
02,0.550404,0.491327,26359.984354,1.120240
03,0.878683,0.593233,0.000000,1.481176
04,0.000047,0.010995,4.157847,0.004254
05,0.002111,0.013125,4.596322,0.160864
06,0.121531,0.280950,3.870047,0.432572
07,0.128173,0.243402,17.245695,0.526589
08,0.002102,0.016565,7.065828,0.126886
09,-0.000534,0.007982,3.636314,-0.066892
10,0.024011,0.089457,4.336548,0.268414


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,12.000000,12.000000
mean,0.178687,0.196075,2204.947883,0.443497
std,0.283413,0.211148,7606.875388,0.485601
min,-0.000534,0.007982,0.000000,-0.066892
25%,0.001940,0.013500,3.811613,0.121874
50%,0.031123,0.125646,4.466435,0.252337
75%,0.195635,0.318314,9.610795,0.626133
max,0.878683,0.593233,26359.984354,1.481176


In [0]:
index_2010 =  find_montly_index('2010')
res_2010_mi = dict()
for mo in index_2010.keys():
    start, end = min(index_2010[mo]), max(index_2010[mo])
    portfolio = track_portfolio_performance_1(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", "mmd"]
    res_2010_mi[mo] = []
    res_2010_mi[mo].append(portfolio_df['ret'].mean() - r)
    res_2010_mi[mo].append(portfolio_df['ret'].std())
    res_2010_mi[mo].append(portfolio_df['mmd'].max())


In [112]:
df_2010_2 = pd.DataFrame(res_2010_mi, index=['average excess returns', 'vol', 'mmd']).T
df_2010_2['sharpe ratio'] = df_2010_2['average excess returns'] / df_2010_2['vol']
display(df_2010_2)
display(df_2010_2.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.024896,0.050912,4.299144,0.489007
02,0.403144,0.321397,0.000000,1.254347
03,0.098089,0.183128,2.187180,0.535634
04,0.063886,0.033109,2.177967,1.929548
05,0.241920,0.335342,0.000000,0.721411
06,0.154932,0.102090,0.000000,1.517607
07,0.401760,0.807520,227.573408,0.497523
08,0.030106,0.052722,3.730956,0.571040
09,0.253840,0.972971,28.289371,0.260891
10,0.042600,0.040386,2.354573,1.054812


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,12.000000,12.000000
mean,0.149332,0.248702,23.050141,0.887004
std,0.142479,0.320218,64.863365,0.486937
min,0.024896,0.033109,0.000000,0.260891
25%,0.039360,0.042748,1.633475,0.526106
50%,0.080988,0.077406,2.527950,0.784250
75%,0.244900,0.324883,3.873003,1.104695
max,0.403144,0.972971,227.573408,1.929548


In [0]:
index_2011 =  find_montly_index('2011')
res_2011_mi = dict()
for mo in index_2011.keys():
    start, end = min(index_2011[mo]), max(index_2011[mo])
    portfolio = track_portfolio_performance_1(start, end)
    portfolio_df = pd.DataFrame.from_dict(portfolio, orient='index')
    portfolio_df.columns = ["ret", "pnl", "mmd"]
    res_2011_mi[mo] = []
    res_2011_mi[mo].append(portfolio_df['ret'].mean() - r)
    res_2011_mi[mo].append(portfolio_df['ret'].std())
    res_2011_mi[mo].append(portfolio_df['mmd'].max())


In [114]:
df_2011_2 = pd.DataFrame(res_2011_mi, index=['average excess returns', 'vol', 'mmd']).T
df_2011_2['sharpe ratio'] = df_2011_2['average excess returns'] / df_2011_2['vol']
display(df_2011_2)
display(df_2011_2.describe())

,average excess returns,vol,mmd,sharpe ratio
01,0.057832,0.037538,2.048088,1.540632
02,0.048800,0.038866,0.573225,1.255595
03,0.491876,0.684476,13.306305,0.718616
04,0.156944,0.298502,7.279768,0.525774
05,0.081969,0.132050,0.833833,0.620741
06,0.061895,0.049576,1.264135,1.248473
07,0.487266,1.084882,890.936832,0.449142
08,0.289143,0.187886,0.000000,1.538930
09,0.443477,0.327623,287.772652,1.353621
10,0.187054,0.625567,33.210668,0.299015


,average excess returns,vol,mmd,sharpe ratio
count,12.000000,12.000000,12.000000,12.000000
mean,0.253187,0.439457,103.588997,0.852842
std,0.213515,0.457282,260.973415,0.495718
min,0.048800,0.037538,0.000000,0.244591
25%,0.073068,0.111432,0.890227,0.446601
50%,0.171999,0.306233,3.490763,0.669679
75%,0.454424,0.640294,18.282396,1.280102
max,0.655196,1.492550,890.936832,1.540632


By observation, considering the market impact results in reduced average returns a lot (from 23% to 14%) and increased vol (aounnd 50%). Regarding the maximum drawdown value, penalize the market impact also reduced the maximum draw down significantly. The overall performance (sharpe ratio) gets worst, becasue reducing the average excess return is the most dominant force.